<a target="_blank" href="https://colab.research.google.com/github/james-trayford/dotAstro13/blob/main/strauss_dotAstro13.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

Notebook prepared by **Dr James Trayford** - for queries please email [`james.trayford@port.ac.uk`](mailto:james.trayford@port.ac.uk)

To use this notebook (if you haven't already) you can first save a copy to your local drive by clicking `File > Save a Copy in Drive` and run on that copy. `Edit > Clear all outputs` on that copy should also ensure you have a clean version to start from.

# **.Astronomy 13** - 🔊🎶🎵 Sonification of data with `strauss` 

In this notebook, I've tried to make the different examples stand-alone so you can run and re-run cells out of order once you've got the data. This means repeating code between cells. To give some hints at what you might want to edit in these examples, I've added comments and the "✏️" symbol ""

first, let's install the `strauss` code!

In [ ]:
 %pip install --quiet git+https://github.com/james-trayford/strauss.git@spectraliser_update

*We will use the `spectraliser` development branch for this notebook. Install can take a while - but you should only need to run it once!*


We'll also grab the repository from _GitHub_ for easy access to some files

In [ ]:
 !git clone https://github.com/james-trayford/strauss.git

Now, let's import the modules we need...

In [ ]:
%reload_ext autoreload 
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
from strauss.sonification import Sonification
from strauss.sources import Objects, Events
from strauss import channels
from strauss.score import Score
from strauss.generator import Synthesizer, Spectralizer, Sampler
from strauss import sources as Sources

import IPython.display as ipd
import os
from scipy.interpolate import interp1d
import numpy as np
import requests

# modules to display in-notebook
import IPython.display as ipd
from IPython.display import display, Markdown, Latex, Image

## Section 1 - Getting Data

⚠  ***The cells in this section are to organise the data we need for the session, don't worry too much about the details of the code here!*** ⚠

In [ ]:
import glob
import urllib
import zipfile
import os

We download and unpack the data used in this notebook:

In [ ]:
# Don't bother if the directory already exists (i.e. Section 0 was run)
if not os.path.isdir('prepared_data'):
    outdir = "./prepared_data"

    path = os.path.realpath(outdir)
    if not glob.glob(outdir):
      os.mkdir(path)

    fname = "dotAstro13_STRAUSS_tutorial.zip"
    url = "https://drive.google.com/uc?export=download&id=1sXesBtj2DE1fyThay8EX6l87H9cDz0-h"
    print(f"Downloading files...")

    with urllib.request.urlopen(url) as response, open(f"{path}/{fname}", 'wb') as out_file:
      data = response.read() # a `bytes` object
      out_file.write(data)

    print(f"Unzipping files to {outdir}/ ...")
    with zipfile.ZipFile(f"{outdir}/{fname}", 'r') as zip_ref:
        zip_ref.extractall(f"{outdir}")

    print(f"Clearing up...")
    os.remove(f"{path}/{fname}")

    print("Done.")

First, we organise the data:

In [ ]:
lc = {}
psd = {}

for f in glob.glob('prepared_data/variable_stars/*_lc.dat'):
    tag = f.split(os.sep)[-1].split('_')[0]
    lc[tag] = np.genfromtxt(f)
    psd[tag] = np.genfromtxt(f'prepared_data/variable_stars/{tag}_psd.dat')  

Define a useful function for plotting these lightcurves as we deal with them...

In [ ]:
def show_lightcurve(name, plot_type='scatter', truncate_index=None): 
    x = lc[name][:N,0]-lc[name][:N,0].min()
    y = lc[name][:N,1]

    if plot_type == 'scatter':
        plt.scatter(x, y, s=4)
    if plot_type == 'line':
        plt.plot(x, y)
        
    plt.ylabel('Flux')
    plt.xlabel('Time [days]')
    plt.title(name)
    plt.show()
    return x, y 

Show what targets we've got...

In [ ]:
targets = list(psd.keys())
print(targets)

...and pick which we're going to sonify using the different variations below

In [ ]:
targlist = ['55Cancri'] # Pick an object or iterate through all 'targets'  ✏️
#targlist = targets

Now, we are ready iterate through target light-curves to sonify them, alongside their plots!

## Section 2 - One-dimensional data series

Let's consider the simple case of one-dimensional data series. In these examples, we have light-curves and periodograms from a number of diverse variable stars. How can we go about sonifying them?

Here we will sonify light-curves as a ***one-dimensional time series*** , where some ***sound property*** is is varied with ***time*** in the ***sonification***, in the same way that **flux**, varies with ***observation time*** in a ***light-curve*** (early in the sonification is shorter wavelengths, later is longer wavelengths).

### Section 2.1 - Event sonification vs Scatter Plots

In `strauss` we could treat each ***flux density*** data point in the spectra as separate audio `Events`, with an occurence `time` mapped from their ***wavelength***.

However, articulating each data point as a separate ***note*** for ***many thousands*** of data points can require long and drawn-out sonifications.

Here we demonstrate this approach with just a portion of the data points (staying within `Colab`'s RAM limitations for unpaid users 🤫). We use the `Synthesizer` object with the `pitch_mapper` preset by default - this has a default pitch range of ***two octaves*** (a factor of 4 in frequency) and we pick an `E3` note (165 Hz) as the base (lowest) frequency.

We will hear the **flux** of each point as `pitch`, with their observation mapped to the occurence `time` (moving from low to high wavelength). 

In [ ]:
# set an index to trunate the lightcurve (if it's too long...)
N = 150
#N = None


for trg in targlist: # iterate, if you like
    
    x,y = show_lightcurve(trg, plot_type='scatter', truncate_index=N)
    
    # specify the base notes used. In this example we use a single E3 note and
    # freely vary the pitch via the 'pitch_shift' parameter
    notes = [["E3"]]
    
    # we could also just specify a particular frequency... ✏️
    # notes = [[150.]]
    
    score =  Score(notes, 15)
    
    maps = {'pitch':np.ones(x.size),
            'time': x,
            'pitch_shift':y}
    
    # specify audio system (e.g. mono, stereo, 5.1, ...)
    system = "mono"
    
    # set up synth (this generates the sound using mathematical waveforms)
    generator = Synthesizer()
    generator.load_preset('pitch_mapper')
    
    # or maybe the sampler instead by uncommenting this block (this uses recorded audio clips)
    # generator = Sampler(sampfiles="./strauss/data/samples/mallets")
    
    generator.modify_preset({'note_length':0.1,
                             'volume_envelope': {'use':'on',
                                                 # A,D,R values in seconds, S sustain fraction from 0-1 that note
                                                 # will 'decay' to (after time A+D)
                                                 'A':0.02,    # ✏️ for such a fast sequence, using ~10 ms values
                                                 'D':0.02,    # ✏️ for such a fast sequence, using ~10 ms values
                                                 'S':0.,      # ✏️ decay to volume 0
                                                 'R':0.001}}) # ✏️ for such a fast sequence, using ~10 ms values
    
    # set 0 to 100 percentile limits so the full pitch range is used...
    # setting 0 to 101 for pitch means the sonification is 1% longer than the final note position
    lims = {'time': ('0','101'),
            'pitch_shift': ('0','100')}
    
    # set up source
    sources = Events(maps.keys())
    sources.fromdict(maps)
    sources.apply_mapping_functions(map_lims=lims)
    
    soni = Sonification(score, sources, generator, system)
    soni.render()
    # soni.save('pitchpm.wav')
    dobj = soni.notebook_display(show_waveform=0);

In [ ]:
# set an index to trunate the lightcurve (if it's too long...)
N = 950
#N = None

for trg in targlist: # iterate, if you like
    
    x,y = show_lightcurve(trg, plot_type='scatter', truncate_index=N)
    
    # specify the notes used.
    # C major pentatonic
    notes = [["C3","E3","F3","G3","B3","C4","E4","F4","G4","B4","C5","E5","F5","G5","B5"]]
    
    # or a whole-tone scale, (was it just a dream... a dream... a dream...)
    #
    # This is a "symmetrical" scale and here also demonstrates using raw frequencies in Hz.
    # The whole tone scale uses 2 semitone jumps, what about minor thirds (3) fourths (5)
    # or fifths (7)? Specify `semitones` to change this.
    # `nint` specifies the number of intervals in the scale. Note a bigger semitone interval
    # will reach higher pitches (perhaps inaudible ones...) ✏️
    
    # semitones = 2.
    # nint = 10
    # notes = [100*2**(np.arange(nint)*(semitones/12))]
    
    # ------
    # In this context, we also consider the `Score` optional parameter `pitch_binning`.
    #
    # This determines how we bin the 'pitch' quantity, which can be either:
    # - 'uniform':  the range of mapped values for 'pitch' are split into even bins
    #               representing  each of the scored notes
    # - 'adaptive': the range of mapped values for 'pitch' split by percentile.
    #               such that each interval is played approximately the same number of
    #               times
    # without specifying,  'adaptive' is used by default, but we specify 'uniform' here
    #
    # which is better for representing this data?
    
    # we specify 'uniform' pitch binning in the primary example... ✏️
    score =  Score(notes, 15, pitch_binning='uniform')
    
    # what about adaptive?
    #score =  Score(notes, 15, pitch_binning='adaptive')
    
    maps = {'pitch':y,
            'time': x}
    
    # specify audio system (e.g. mono, stereo, 5.1, ...)
    system = "mono"
    
    # set up synth (this generates the sound using mathematical waveforms)
    generator = Synthesizer()
    generator.load_preset('pitch_mapper')
    
    generator.modify_preset({'note_length':0.15,
                             'volume_envelope': {'use':'on',
                                                 # A,D,R values in seconds, S sustain fraction from 0-1 that note
                                                 # will 'decay' to (after time A+D)
                                                 'A':0.02,    # ✏️ for such a fast sequence, using ~10 ms values
                                                 'D':0.02,    # ✏️ for such a fast sequence, using ~10 ms values
                                                 'S':1.,      # ✏️ decay to volume 0
                                                 'R':0.001}}) # ✏️ for such a fast sequence, using ~10 ms values
    
    # set 0 to 100 percentile limits so the full pitch range is used...
    # setting 0 to 101 for pitch means the sonification is 1% longer than
    # the time needed to trigger each note - by making this more than 100%
    # we give all the notes time to ring out (setting this at 100% means
    # the final note is triggered at the momement the sonification ends)
    lims = {'time': ('0','101'),
            'pitch_shift': ('0','100'),
            'pitch': ('0','100')}
    
    # set up source
    sources = Events(maps.keys())
    sources.fromdict(maps)
    sources.apply_mapping_functions(map_lims=lims)
    
    soni = Sonification(score, sources, generator, system)
    soni.render()
    dobj = soni.notebook_display(show_waveform=0)

The articulation of each note lets you hear each data point separately, but over many noisy data points can lead to a confusing result - it's hard to keep track of the fluxes and our pitch memory is challenged.

Instead we can try ***smoothly evolving*** parameters, designating the spectra as an ***evolving `Object`*** source class. We demonstrate this in the following subsection `3.1`.

### Section 2.2: Object Sonification vs Line Plots

Let's first try the evolving Object approach without truncating the raw data in any way. In analogy to visual display, the Event representation is like plotting the spectrum as a scatter plot, while an Object representation is like plotting the spectrum as a continuous line.

let's hear the 1D time-series data sonification, mapping flux density to pitch for the spectrum.

*NB: If you were wondering why `strauss` refers to the varying pitch mapping as `pitch_shift` and not just `pitch`, this is because all sources in `strauss` also need a base `pitch` which is chosen from the `'notes'` structure (here always `'A2'`) by the `Score`. This is because `strauss` can play many sources at the same time! Again, you can read more about this [in the docs](https://strauss.readthedocs.io/en/latest/).*

In [ ]:
# # show spectrum again, for reference
# display(Image(spectra_plots[stype][slabel]))

for trg in targlist: # iterate, if you like  
    # set an index to truncate the lightcurve (if it's too long...)
    N = None
    x,y = show_lightcurve(trg, plot_type='line')

    notes = [["A2"]]
    score =  Score(notes, 15)
    
    # set up synth (this generates the sound using mathematical waveforms)
    generator = Synthesizer()
    generator.preset_details('pitch_mapper')
    generator.load_preset('pitch_mapper')
    
    
    data = {'pitch':1.,
            'time_evo':x,
            'pitch_shift':y}
    
    # set 0 to 100 percentile limits so the full pitch and time range is used...
    lims = {'time_evo': ('0','100'),
            'pitch_shift': ('0','100')}
    
    # set up source
    sources = Objects(data.keys())
    sources.fromdict(data)
    sources.apply_mapping_functions(map_lims=lims)
    
    soni = Sonification(score, sources, generator, system)
    soni.render()
    dobj = soni.notebook_display(show_waveform=0);

How about mapping a low-pass filter to flux density?

*using a 'tonal' carrier, uncomment line for a 'textural' (or white noise) carrier*

In [ ]:
for trg in targlist: # iterate, if you like

    # set an index to trucnate the lightcurve (if it's too long...)
    N = None
    x,y = show_lightcurve(trg, plot_type='line', truncate_index=N)
    
    generator = Synthesizer()
    generator.modify_preset({'filter':'on'})
    
    # uncomment these lines to try a 'textural' sonification using white noise! ✏️
    # generator.load_preset('windy')
    # generator.preset_details('windy')
    
    # we use a 'chord' here to create more harmonic richness (stacking fifths)...
    notes = [["A2", "E3", 'B3', 'F#4']]
    score =  Score(notes, 15)
    
    data = {'pitch':[0,1,2,3],
            'time_evo':[x]*4,
            'cutoff':[y]*4}
    
    lims = {'time_evo': ('0','100'),
            'cutoff': ('0','100')}
    
    # set up source
    sources = Objects(data.keys())
    sources.fromdict(data)
    plims = {'cutoff': (0.15,0.95)}
    sources.apply_mapping_functions(map_lims=lims, param_lims=plims)
    
    soni = Sonification(score, sources, generator, system)
    soni.render()
    dobj = soni.notebook_display(show_waveform=0);

In fact, there are many expressive properties of sound we could use to represent the data in a similar way. In `strauss` these are referred to as `mappable` properties.

A subset of these can be used as an evolving property with the `Object` source class. These are referred to as `evolvable` properties.

Lets show what's available:

In [ ]:
display(Markdown(f"### ***'Mappable'*** properties:"))
for m in Sources.mappable:
  display(Markdown(f' * `{m}` '))

display(Markdown(f"### ***'Evolvable'*** properties:"))
for m in Sources.evolvable:
  display(Markdown(f' * `{m}` '))

We can play around with some of these `evolvable` properties here. Are all of these effective for this data? Could they be effective for other types of data representations?

The `idx` variable below controls which evolvable property is selected from the `some_mappings` list. `idx` can be changed to a number from 0 to 5 inclusive to choose a certain sound parameter from the list.

In [ ]:

for trg in targlist: # iterate, if you like

    # set an index to trunate the lightcurve (if it's too long...)
    N = None
    x,y = show_lightcurve(trg, plot_type='line', truncate_index=N)
    
    # A list of some 'evolvable' mappings
    some_mappings = ["volume",
                     "azimuth",
                     "volume_lfo/amount",
                     "volume_lfo/freq_shift",
                     "pitch_lfo/amount",
                     "pitch_lfo/freq_shift"]
    
    # change this (between 0 and 5) to select a different property to map...
    idx = 3
    
    # use a stereo system to allow 'phi' mapping (low pan left and high pan right)
    system = "stereo"
    
    display(Markdown(f"### Sonifying in 1D using `evolvable` property - ***`{some_mappings[idx]}`***:"))
    
    generator = Synthesizer()
    generator.modify_preset({'filter':'on',
                             "pitch_hi":-1, "pitch_lo": 1,
                             "pitch_lfo": {"use": "on",
                                           "amount":1*("pitch_lfo/freq_shift" == some_mappings[idx]),
                                           "freq":3*5**("pitch_lfo/freq_shift" != some_mappings[idx]),
                                           "phase":0.25},
                             "volume_lfo": {"use": "on",
                                            "amount":1*("volume_lfo/freq_shift" == some_mappings[idx]),
                                            "freq":3*5**("volume_lfo/freq_shift" != some_mappings[idx]),
                                            "phase":0}
                            })
    
    # try a different chord (stacking fifths)...
    notes = [["A2","E3","B4","F#4"]]
    
    # chords and can also be specified via chord names and base octave....
    # notes = "Em6_3"
    
    
    score =  Score(notes, 15)
    
    data = {'pitch':[0,1,2,3],
            'time_evo':[x]*4,
            'cutoff':[0.9]*4,
            'polar':[0.5]*4,
            some_mappings[idx]:[(y-y.min())/(y.max()-y.min())]*4}
    
    lims = {'time_evo': ('0','100'),
            "volume": ('0','100'),
            "azimuth": (-0.5,1.5),
            "volume_lfo/amount": ('0','100'),
            "volume_lfo/freq_shift": ('0','100'),
            "pitch_lfo/amount": ('0','100'),
            "pitch_lfo/freq_shift": ('0','100')}
    
    # set up source
    sources = Objects(data.keys())
    sources.fromdict(data)
    plims = {'cutoff': (0.25,0.9)}
    sources.apply_mapping_functions(map_lims=lims, param_lims=plims)
    
    soni = Sonification(score, sources, generator, system)
    soni.render()
    dobj = soni.notebook_display(show_waveform=0.);

## 4. Sonification: Spectral Representation

Having covered some of these more abstract approaches, let's consider a direct _Spectral Audification_ or _"Spectralisation"_

We will try converting the ***periodogram*** into a ***sound spectrum*** directly, i.e. the sound has the same frequency features as the observed light curves, albeit at ***audible frequencies***

In [ ]:
for c in targlist:

    fmin = 0.2
    fmax = 20  
    
    freqs = psd[c][:,0].copy()
    fsel = np.logical_and(freqs < fmax, freqs > fmin) 
    powers = psd[c][:,1][fsel]
    freqs = freqs[fsel]

    plt.plot(freqs, powers)
    plt.xlabel(r'Frequency [days$^{-1}$]')
    plt.ylabel('Power')
    plt.title(c)
    plt.show() 

    dynrange = freqs.max()/freqs.min()
    
    # set up spectralizer generator
    generator = Spectralizer()
    
    # Lets pick the mapping frequency range for the spectrum... ✏️
    generator.modify_preset({'min_freq':100, 'max_freq':100*dynrange})
    
    score =  Score([['A2']], 10)
    
    # set up spectrum and choose some envelope parameters for fade-in and fade-out
    maps = {'spectrum': [powers], 'pitch':[1],
            'volume_envelope/D':[0.8],
            'volume_envelope/S':[0.],
            'volume_envelope/A':[0.01]}
    
    # again, use maximal range for the mapped parameters
    lims = {'spectrum': ('0','100')}


    # set up source
    sources = Events(maps.keys())
    sources.fromdict(maps)
    sources.apply_mapping_functions(map_lims=lims)
    
    # render and play sonification!
    soni = Sonification(score, sources, generator, system)
    soni.render()
    dobj = soni.notebook_display(show_waveform=0);

Can you hear how all these lightcurves give a different _"timbre"_ or _"formant"_?

For some more context, we take the _"55-Cancri"_ curve in particular.

Let's isolate the fundamental frequency of the short period variation (rapid dips) and some of it's overtones:

In [ ]:
targ = '55Cancri'

fmin = 0.2
fmax = 20 

freqs = psd[targ][:,0].copy()
fsel = np.logical_and(freqs < fmax, freqs > fmin) 
powers = psd[targ][:,1][fsel]
freqs = freqs[fsel]
    
plt.plot(freqs*(100/freqs.min()), powers)
plt.title(targ)
plt.xlim(100, 100*dynrange)
# plt.axvline(115)

harmonics = [660]
cdx = 0
for h in harmonics:
    cdx +=1
    for k in range(1,12):
        plt.axvline(h*k, c=f"C{cdx}",ls=':')
plt.xlabel("Spectral Sonification Frequency distribution [Hz]")
plt.ylabel("Power")
# plt.semilogx()
plt.show()

Turns out the dominant fundamental in _55 Cancri's_ periodogram is sonified at around 660 Hz, or an 'E5' in musical notation, lets hear that for comparison... 

In [ ]:

score =  Score([['E5']], 2)
#score =  Score([[660.]], 2)

maps = {'pitch': [1]}

# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# set up synth (this generates the sound using mathematical waveforms)
generator = Synthesizer()

generator.load_preset('pitch_mapper')
generator.modify_preset({"volume_envelope": {"use": "on", "S": 0., "D": 2}
                        })

# set up source
sources = Objects(maps.keys())
sources.fromdict(maps)
sources.apply_mapping_functions(map_lims=lims)

# render and play sonification!
soni = Sonification(score, sources, generator, system)
soni.render()

soni.notebook_display(show_waveform=False);

# Bonus Material!

Here's some additional examples to explore if you're interested!

## A. "Stars Appearing" from the _Tour of the Universe_ planetarium show

<u> __The Score:__ </u>

We set up the ***Score***; this is analagous to a musical score and controls what notes can be played over the course of the sonification. We can specify a chord sequence as a single string (`str`) where chord names are separated by a `|` character. The root octave of the chord may also be specified by adding `_X` where `X` is the octave number. <span style="color:gray">_(Note: for now, each chord occupies an equal lenth in the sonification, in the future chord change times can be directly specified and optionally related to events in the data)_</span>

We can directly specify the ___chord voicing___ as 
a list of lists containing the notes from low to high in each chord.

Here, we  are directly specifying a single __`Db6/9`__ chord voicing. These notes will later be played by stars of different colours!

In [ ]:
chords = [['Db3','Gb3', 'Ab3', 'Eb4','F4']]
length = "1m 30s"
score =  Score(chords, length)

<u> __The Sources:__ </u>

Next, we import the data that will represent ___sources___ of sound. The data is the sky positions, brightness and colour of stars from the _Paranal_ observatory site in _Chile_. This data is contained in an `ascii` file (specified by the `datafile` variable) and organised where each __row__ is a star and each __column__ is a property of that star.

The idea here is that as night draws in we see the brightet stars first. As it gets darker, and our eyes adjust, we see more dim stars. We "***sonify***" this by having a note play when each star appears. The "***panning***" (a.k.a stereo imaging) of the note is controlled by the ***altitude*** and ***azimuth*** of the stars, as if we were facing south. The ***colour*** of the star contols the note within the chord we've chosen, where notes low to high (short to long wavelength) represent fixed-number bins in colour from blue to red (again, short to long wavelength). Finally, the volume of the note is also related to the brightness of the star (dimmer stars are quieter). This is chosen to give a relatively even volume throught the sonification, as dim stars are much more numerous than bright ones <span style="color:gray">_(Note: in the future we will have the option to scale volumes in this way automatically)_</span>

We speciify the sound preperty to star property mappimg as as three `dict` objects with keys representing each sound property we dapat in the sonification:
- **`mapcols`**: entries are the data file columns used to map each property
- **`mapvals`**: entries are function objects that manipulate each columns values to yield the linear mapping
- **`maplims`**: entries are `tuple`s representing the (`low`,`high`) limits of each mapping.numerical values represent absolute limits (used here for the angles in degrees to correctly limit the `azimuth` and `polar` mappings to 360° (2π) and 180° (π) respectively. `str` values are taken to be percentiles from 0 to 100. string values > 100 can also be used, where e.g. 104 is 4% larger than the 100th percentile value. This is used for the time here, so that the last sample doesnt trigger at exactly the end of the sonification, giving time for the sound to die away slowly.

In [ ]:
datafile = "./strauss/data/datasets/stars_paranal.txt"
mapcols =  {'azimuth':1, 'polar':0, 'volume':2, 'time':2, 'pitch':3}

mapvals =  {'azimuth': lambda x : x,
            'polar': lambda x : 90.-x,
            'time': lambda x : x,
            'pitch' : lambda x: -x,
            'volume' : lambda x : (1+np.argsort(x).astype(float))**-0.2}

maplims =  {'azimuth': (0, 360),
            'polar': (0, 180), 
            'time': ('0', '104'),
            'pitch' : ('0', '100'),
            'volume' : ('0', '100')}

events = Events(mapcols.keys())
events.fromfile(datafile, mapcols)
events.apply_mapping_functions(mapvals, maplims)

<u> __The Generator:__ </u>

The final element we need is a ***Generator*** that actually generates the audio given the ***Score*** and ***Sources***. Here, we use a ***Sampler***-type generator that plays an audio sample for each note. The samples and other parameters (not specified here) control the sound for each note. These can be specified in `dict` format note-by-note (keys are note name strings, entries are strings pointing to the `WAV` format audio sample to load) or just using a string that points to a sample directory (each sample filename in that directory ends with `_XX.wav` where `XX` is the note name) we use the example sample back in `./data/samples/glockenspiels` <span style="color:gray">_(Note: rendering can take a while with the long audio samples we use here, shorter samples can be used to render faster, such as those in `./data/samples/mallets`. This is also useful if you want to try different notes or chords, as only the 5 notes specified above are provided in the glockenspiel sample folder.)_</span>


In [ ]:
sampler = Sampler("./strauss/data/samples/glockenspiels")
sampler.preset_details("default")

<u> __The Sonification:__ </u>

We consolidate the three elements above in to a sonification object to generate the sound, specifying the audio setup (here `'stereo'` as opposed to `'mono'`, `'5.1'`, etc). <span style="color:gray">_(Note: you can generate the audio in any specified audio setup, but following cells assume stereo and only mono and stereo formats are supported by the jupyter audio player in the final cell)_</span>

We then `render` the sonification to generate the audio track (may take some time with the glockenspiel samples).

In [ ]:
system = "stereo"

In [ ]:
soni = Sonification(score, events, sampler, system)
soni.render()

Finally, let's visualise the waveform, and preview the audio in-notebook*!

<span style="color:gray">_*if using a surround sound format (i.e > 2 channels) the preview is stereo, with the first two channels mapped left and right, due to the limitations of the notebook audio player_</span>

In [ ]:
soni.notebook_display()

## B. Bivariate Sonification with EAGLE star formation histories

Lets look at these multivariate data sets. These are star-formation and metal-enrichment histories of 5 galaxies from the [EAGLE simulations](https://eagle.strw.leidenuniv.nl/) (virtual galaxies, modelled on a supercomputer) 

We see that we have two time-dependent quantities: The rate at which stars form (aka ***SFR***, in solar masses per year) and the mass fraction of those stars in elements heavier than Helium (aka ***Metallicity*** or $Z$) 

Let's take a look...

In [ ]:
import pandas as pd

for f in glob.glob('./prepared_data/eagle_galaxies/*.txt'):
  data_table = pd.read_csv(f)
  display(Markdown(f"### File: <mark>`{f}`</mark>"))
  xlab = data_table.columns[0]
  ylab = data_table.columns[1]
  zlab = data_table.columns[2]

# show the last-loaded SFHs
plt.plot(data_table[xlab], data_table[ylab])
plt.xlabel(f'{xlab}')
plt.ylabel(f'{ylab}')
plt.show()
plt.plot(data_table[xlab], data_table[zlab])
plt.xlabel(f'{xlab}')
plt.ylabel(f'{zlab}')
plt.show()

Now let's choose some combination of evolvable parameters!

Now there are 2 properties being sonified at once (not including the time variable), there are 28 possible combinations of these mappings $\left(\frac{8!}{2!(8-2)!} = 28\right)$ 

Some combinations might work better than others...

In [ ]:
%matplotlib inline

# A list of some 'evolvable' mappings
some_mappings = ["pitch_shift",
                 "cutoff",
                 "volume",
                 "phi",
                 "volume_lfo/amount", 
                 "volume_lfo/freq_shift",
                 "pitch_lfo/amount", 
                 "pitch_lfo/freq_shift"]

# !! change these (between 0 and 7) to select a different property to map... 
idx_sfr = 1
idx_Z = 5

chosen = [some_mappings[idx_Z],some_mappings[idx_sfr]]

# use a stereo system to allow 'phi' mapping (low pan left and high pan right)
system = "stereo"

# some strauss setup can happen outside the loop...
generator = Synthesizer()

generator = Synthesizer()
generator.modify_preset({'filter':'on',
                         "pitch_hi":-1, "pitch_lo": 1,
                         "pitch_lfo": {"use": "on", 
                                       "amount":1*("pitch_lfo/freq_shift" in chosen), 
                                       "freq":3*5**("pitch_lfo/freq_shift" not in chosen), 
                                       "phase":0.25},
                         "volume_lfo": {"use": "on", 
                                        "amount":1*("volume_lfo/freq_shift" in chosen), 
                                        "freq":3*5**("volume_lfo/freq_shift" not in chosen), 
                                        "phase":0}
                        })


notes = [["A2", "E3"]]
score =  Score(notes, 30)

lims = {'time_evo': ('0','100'),
        'pitch_shift': ('0','100'),
        'cutoff': ('0','100'),
        "volume": ('0','100'), 
        "phi": (-0.5,1.5),
        "volume_lfo/amount": ('0','100'), 
        "volume_lfo/freq_shift": ('0','100'),
        "pitch_lfo/amount": ('0','100'), 
        "pitch_lfo/freq_shift": ('0','100')}

display(Markdown(f"## Mapping SFR to <mark>***`{some_mappings[idx_sfr]}`***</mark> and Metallicity to <mark>***`{some_mappings[idx_Z]}`***</mark>:"))

for f in glob.glob('./AU2_Group4/Data_bonus/*.txt'):

  display(Markdown(f"### EAGLE Galaxy <mark>***`{f.split('.')[1].split('_')[-1]}`</mark>***:"))

  # get data
  data_table = np.genfromtxt(f, delimiter=',')[1:]
  time = data_table[:,0]
  sfr = data_table[:,1]
  Z = data_table[:,2]

  # plot multivariate data ------------------------ 
  plt.plot(time,sfr)
  plt.xlabel(f'Time [Gyr]')
  plt.ylabel(f'Star Formation Rate [Msun/year]')
  plt.gca().twinx()
  plt.plot([0],[0], label='Star Formation Rate') # dummy plot to make legend work
  plt.plot(time,Z,c='C1',label="Stellar Metallicity")
  plt.xlabel(f'Time [Gyr]')
  plt.ylabel(f'Metallicity')
  plt.legend(frameon=0)
  plt.show();
  # ------------------------------------------------ 

  data = {'pitch':[0,1],
          'time_evo':[time]*2,
          'theta':[0.5]*2,
          some_mappings[idx_sfr]:[(sfr - sfr.min())/(sfr.max()-sfr.min())]*2,
          some_mappings[idx_Z]:[(Z - Z.min())/(Z.max()-Z.min())]*2}

  # set up source
  sources = Objects(data.keys())
  sources.fromdict(data)
  plims = {'cutoff': (0.4,1)}
  sources.apply_mapping_functions(map_lims=lims, param_lims=plims)

  soni = Sonification(score, sources, generator, system)
  soni.render()
  dobj = soni.notebook_display(show_waveform=0)

## C. More _Spectralising_ : Planetary Nebulae and the old STRAUSS logo

In other examples we use a 'parameter mapping' approach for one-dimensional data series, where we map _y_ as a function of _x_ using the change in some expressive property of sound (e.g. `pitch_shift`) as a function of time.

We consider a direct spectralisation approach where the sopund is generated by treating th 1D data as a sound spectrum! This uses a direct inverse Fourier transform.This seems relatively intuitive for spectral data, particular those with spectral features similar to what we can identify in sound.

We will use Planetary Nebulae (PNe) data to demonstrate this, objects that are dominated by strong emission lines...

**First, let's grab some data...**

In [ ]:
spectral_data1 = np.genfromtxt('./strauss/data/datasets/NGC1535.csv', delimiter=',')
wlen1 = spectral_data1[:,0]
fluxdens1 = spectral_data1[:,1]

spectral_data2 = np.genfromtxt('./strauss/data/datasets/NGC6302.csv', delimiter=',')
wlen2 = spectral_data2[:,0]
fluxdens2 = spectral_data2[:,1]

# spectrum needs to be provided to the Spectraliser in frequency order (i.e. low to high), 
# so we ensure it is sorted that way... 
spec1 = fluxdens1[np.argsort(1/wlen1)]
spec2 = fluxdens2[np.argsort(1/wlen2)]
wlen1 = wlen1[np.argsort(1/wlen1)]
wlen2 = wlen2[np.argsort(1/wlen2)]

# plot the spectra vs wavlength
plt.plot(wlen1, spec1/spec1.max(), label= 'NGC1535')
plt.plot(wlen2, spec2/spec2.max(), alpha=0.6, label= 'NGC6302')
plt.xlabel('Wavelength [Angstrom]')
plt.ylabel('Flux')
plt.legend(frameon=False)
plt.show()

# plot the spectra vs frequency
plt.plot(1e-12*3e8/(wlen1*1e-10), spec1/spec1.max(),label= 'NGC1535')
plt.plot(1e-12*3e8/(wlen2*1e-10), spec2/spec2.max(), alpha=0.6,label= 'NGC1535')
plt.xlabel('Frequency [THz]')
plt.ylabel('Flux')
plt.legend(frameon=False)
plt.show()

**Set up some universal sonification parameters and classes for the examples below**

For all examples we use the `Synthesizer` generator to create a 30 second, mono sonification.

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

# length of the sonification in s
length = 10.

### <u>Example 1</u> &nbsp; **Comparing the NGC 1535 & NGC 6302 Spectra**

Lets compare the _Spectraliser_ representations of these two spectra

In [ ]:
notes = [["A2"]]

score =  Score(notes, length)

spectra = [spec1, spec2]
wlens = [wlen1, wlen2]
names = ['NGC 1535', 'NGC 6302']

for i in range(2):

    #set up spectralizer generator
    generator = Spectralizer()

    # Lets pick the mapping frequency range for the spectrum...
    generator.modify_preset({'min_freq':100, 'max_freq':1000})

    s = np.zeros(spec1.size)
    s[-1] = 1
    # set up spectrum and choose some envelope parameters for fade-in and fade-out
    data = {'spectrum':[spectra[i]], 'pitch':[1],
            'volume_envelope/D':[0.9], 
            'volume_envelope/S':[0.], 
            'volume_envelope/A':[0.05]}
    
    # again, use maximal range for the mapped parameters
    lims = {'spectrum': ('0','100')}
    
    # set up source
    sources = Events(data.keys())
    sources.fromdict(data)
    sources.apply_mapping_functions(map_lims=lims)
    
    # render and play sonification!
    soni = Sonification(score, sources, generator, system)
    soni.render()
    print(f"Spectralising {names[i]}...")
    plt.plot(1e-12*3e8/(wlens[i]*1e-10), spectra[i]/spectra[i].max(), alpha=0.7,label=names[i])
    soni.notebook_display(show_waveform=0)
plt.xlabel('Frequency [THz]')
plt.ylabel('Flux')
plt.legend(frameon=False)
plt.show()

What differences do you notice about the sounds? Can you here the presence/absence of spectral lines, and their relative pitches?

### <u>Example 2</u> &nbsp; **Evolving Spectra and Image Sonification**

We could also perform a `Object` type sonification with an evolving Spectrum. 

An evolving spectrum can be represented as a 2D array, similar to a regular image. Using this similarity, the `Spectraliser` provides a neat way to sonify images!

Here we sonify the `strauss` logo, lets grab it...

In [ ]:
image = plt.imread('./strauss/misc/strauss_logo.png')
image = image[:,:,:-1].sum(axis=-1)
image_inv = 1-image
plt.imshow(image_inv, cmap='gray_r')
plt.axis('off')

in `strauss` each row represents a spectrum, ordered from first to last.

Convention to represent the image is to evolve from left to right, with higher features in the _y_-axis sounding higher pitch. Due to image formatting conventions being different, we need transpse and flip the image array to get the right format for `strauss`

In [ ]:
spec_stack = image_inv[::-1].T
plt.imshow(spec_stack,  cmap='gray_r')
plt.axis('off')

Now let's _Spectralise_!  

In [ ]:
#show the image again...
plt.imshow(image_inv, cmap='gray_r')
plt.axis('off')
plt.show()

score =  Score(notes, 15)

#set up spectralizer generator
generator = Spectralizer()

# Lets pick the mapping frequency range for the spectrum...
generator.modify_preset({'min_freq':20, 'max_freq':10000})

# set up spectrum

# I'm also adding a linear pan from left (azimuth=0.25) to right (azimuth=0.75)
# in the horizontal plane (polar=0.5) to give a sense of scrolling left to right...
data = {'spectrum':[spec_stack], 'pitch':[1], 
        'time_evo': [np.linspace(0,1,1920)],
        'azimuth':[np.linspace(0.25,0.75,1920)],
        'polar':[0.5]}

# again, use maximal range for the mapped parameters
lims = {'spectrum': ('0','100')}

# set up source
sources = Events(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions(map_lims=lims)

# render and play sonification!
soni = Sonification(score, sources, generator, system)
soni.render()
print(f"Spectralising Image...")
soni.notebook_display(show_waveform=0)